<a href="https://colab.research.google.com/github/TongSii/hds5210-2025/blob/main/week08inclass/Copy_of_module26_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parsing HTML with BeautifulSoup

In this example, we want to look at a website and get a list of all the available downloadable files from that website.

https://catalog.data.gov/dataset?res_format=CSV&tags=hospital

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
r = requests.get('https://catalog.data.gov/dataset?res_format=CSV&tags=hospital')

In [ ]:
r.status_code

In [ ]:
print(r.text[0:1000])

In [ ]:
soup = BeautifulSoup(r.text)

In [ ]:
for link in soup.find_all('h3'):
    print(link.a.text)

Non-federal Acute Care Hospital Health IT Adoption and Use
Visits to physician offices, hospital outpatient departments, and hospital emergency departments, by age, sex, and race: United States
Hospitals
Hospital Annual Financial Disclosure Report – Complete Data Set
Hospital Annual Financial Data - Selected Data & Pivot Tables
MD COVID-19 Total Currently Hospitalized Adult and Pediatric Acute and ICU
Pharmacy Medical Billing Services (SV4) Detail Information
Professional Medical Billing Services (SV1) Detail Information - Historical
Initial injury-related visits to hospital emergency departments, by sex, age, and intent and mechanism of injury: United States
Independent dispute resolution summary data
Hospital Quarterly Financial & Utilization Report - Complete Data Set
Professional Medical Billing Services (SV1) Header Information
Hospital Areas
Nursing Homes
GNIS: Buildings (2013)
United States COVID-19 Community Levels by County as Originally Posted
Hospital Dashboard
Automated Ext

In [ ]:
for element in soup.find_all('li', 'dataset-item'):
  print(element.text)





Federal



Non-federal Acute Care Hospital Health IT Adoption and Use



U.S. Department of Health & Human Services —
The adoption and use of electronic health records is a key ONC priority. This page provides data access and documentation for the non-federal acute care hospital EHR...



CSV








Federal



Visits to physician offices, hospital outpatient departments, and hospital emergency departments, by age, sex, and race: United States



U.S. Department of Health & Human Services —
Data on visits to physician offices, hospital outpatient departments and hospital emergency departments by selected population characteristics. Please refer to the...



CSV


RDF


JSON


XML







Hospitals



District of Columbia —
Hospitals. This dataset contains points representing hospital locations, created as part of the DC Geographic Information System (DC GIS) for the D.C. Office of the...



HTML


ArcGIS GeoServices REST API


OGC WMS


CSV


ZIP


GeoJSON


1 more in dataset








In [ ]:
for element in soup.find_all('li', 'dataset-item'):
  print(element.h3.text)


Non-federal Acute Care Hospital Health IT Adoption and Use



Visits to physician offices, hospital outpatient departments, and hospital emergency departments, by age, sex, and race: United States



Hospitals



Hospital Annual Financial Disclosure Report – Complete Data Set



Hospital Annual Financial Data - Selected Data & Pivot Tables



MD COVID-19 Total Currently Hospitalized Adult and Pediatric Acute and ICU



Pharmacy Medical Billing Services (SV4) Detail Information



Professional Medical Billing Services (SV1) Detail Information - Historical



Initial injury-related visits to hospital emergency departments, by sex, age, and intent and mechanism of injury: United States


Independent dispute resolution summary data


Hospital Quarterly Financial & Utilization Report - Complete Data Set


Professional Medical Billing Services (SV1) Header Information


Hospital Areas


Nursing Homes


GNIS: Buildings (2013)


United States COVID-19 Community Levels by County as Originally 

In [ ]:
for element in soup.find_all('li', 'dataset-item'):
    name = element.h3.text.strip()
    resources = element.ul
    #print(resources)
    for item in resources.find_all('li'):
        if item.text.strip() == 'CSV':
            print("Download information about '{}' from {}".format(name,item.a.attrs['href']))#attributes




Download information about 'Non-federal Acute Care Hospital Health IT Adoption and Use' from https://www.healthit.gov/sites/default/files/data/datasets/aha.csv
Download information about 'Visits to physician offices, hospital outpatient departments, and hospital emergency departments, by age, sex, and race: United States' from https://data.cdc.gov/api/views/xt86-xqxz/rows.csv?accessType=DOWNLOAD
Download information about 'Hospitals' from https://opendata.dc.gov/api/download/v1/items/6c18bb76d8644bc1bf53cac2d2199564/csv?layers=4
Download information about 'Hospital Annual Financial Data - Selected Data & Pivot Tables' from https://data.chhs.ca.gov/dataset/ea0c8ca9-023e-46a3-b95b-b9d4ab8ec195/resource/81602900-aa04-4fcb-b70b-19bc0553f79b/download/selected-file_2012_2017_machinereadable.csv
Download information about 'MD COVID-19 Total Currently Hospitalized Adult and Pediatric Acute and ICU' from https://opendata.maryland.gov/api/views/5dbi-7wz2/rows.csv?accessType=DOWNLOAD
Download inf

# Getting Table Data


In this example, we're going to find an HTML table and extract the data from that table https://fhir.epic.com/Specifications?api=981 - Error Codes

In [ ]:
r = requests.get('https://fhir.epic.com/Specifications?api=981')

In [ ]:
r.status_code

200

In [ ]:
soup = BeautifulSoup(r.text)

In [ ]:
# In HTML tables, there is usually a <thead> section to tell us what the column headers are.
# Let's load those into a simple list of headers[]
for table in soup.find_all('table', {'class': 'api-documentation-table'}):
#     print("Found a table")

    headers = []
    for header in table.find_all('th'):
        headers.append(header.text)
headers

['Error Code', 'Severity', 'Description', 'Example']

In [ ]:
# {
#   '4100': ['Fatal', 'The resource request contained an invalid parmater', 'example']
# }

output = {}

for table in soup.find_all('table', {'class': 'api-documentation-table'}):
#     print("Found a table")

    headers = []
    for header in table.find_all('th'):
        headers.append(header.text)

    if headers == ['Error Code', 'Severity', 'Description', 'Example']:
        for row in table.find_all('tr'):
            data = [x.text for x in list(row.find_all('td'))]
            if len(data) != 0:
                code = data[0]
                severity = data[1]
                description = data[2]
                example = data[3]
                output[code] = [severity, description, example]



{
    "4100": [
        "Fatal",
        "The resource request contained an invalid parameter",
        "Invalid parameter such as a nonexistent patient ID: AllergyIntolerance?patient=foo"
    ],
    "4101": [
        "Warning",
        "Resource request returns\u00a0no results",
        "A request for data that was otherwise valid but no information was documented or found (i.e. a patient with no pertinent implanted devices, or a demographic search where no patients met the search criteria)."
    ],
    "4102": [
        "Fatal",
        "The read resource request contained an invalid ID",
        "Invalid Resource ID: AllergyIntolerance/foo"
    ],
    "4103": [
        "Fatal",
        "The resource requested has been deleted",
        "A read request for a resource that was deleted in the system"
    ],
    "4104": [
        "Fatal",
        "A required FHIR element was not available to send in the response",
        "Failed to determine a SNOMED code for the smoking status."
    ]

In [ ]:
import json

print(json.dumps(output, indent=4))

In [ ]:
print(json.dumps(output, indent=4))

{
    "4100": [
        "Fatal",
        "The resource request contained an invalid parameter",
        "Invalid parameter such as a nonexistent patient ID: AllergyIntolerance?patient=foo"
    ],
    "4101": [
        "Warning",
        "Resource request returns\u00a0no results",
        "A request for data that was otherwise valid but no information was documented or found (i.e. a patient with no pertinent implanted devices, or a demographic search where no patients met the search criteria)."
    ],
    "4102": [
        "Fatal",
        "The read resource request contained an invalid ID",
        "Invalid Resource ID: AllergyIntolerance/foo"
    ],
    "4103": [
        "Fatal",
        "The resource requested has been deleted",
        "A read request for a resource that was deleted in the system"
    ],
    "4104": [
        "Fatal",
        "A required FHIR element was not available to send in the response",
        "Failed to determine a SNOMED code for the smoking status."
    ]

In [ ]:
output.get('4118')

['Fatal',
 'User not authorized for request',
 "Request data that the authenticated user is not allowed access to view (i.e. a patient asking for data about a stranger's allergies)."]

## Reading HTML Tables with Pandas

Pandas has the ability to read HTML, too.  In ideal circumstances, it will scour whatever page you give it and find all of the tables there.  The result from `read_html()` will be a list of dataframes.

https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.read_html.html

In [ ]:
import pandas as pd

tables = pd.read_html('https://fhir.epic.com/Specifications?api=981')

In [ ]:
tables

[   Name  Description  Is Optional  Is Array
 0  (  )          NaN          NaN       NaN,
          Name  Description  Is Optional     Is Array  Unnamed: 4
 0  ()  ()  ()          NaN          NaN  conditional         NaN,
          Name  Description  Is Optional     Is Array  Unnamed: 4
 0  ()  ()  ()          NaN          NaN  conditional         NaN,
          Name  Description  Is Optional     Is Array  Unnamed: 4
 0  ()  ()  ()          NaN          NaN  conditional         NaN,
          Name  Description  Is Optional     Is Array  Unnamed: 4
 0  ()  ()  ()          NaN          NaN  conditional         NaN,
    Name  Description  Is Optional  Is Array
 0  (  )          NaN          NaN       NaN,
          Name  Description  Is Optional     Is Array  Unnamed: 4
 0  ()  ()  ()          NaN          NaN  conditional         NaN,
          Name  Description  Is Optional     Is Array  Unnamed: 4
 0  ()  ()  ()          NaN          NaN  conditional         NaN,
    Name  Descriptio

In [ ]:
len(tables)

11

In [ ]:
tables[1]

,Name,Description,Is Optional,Is Array,Unnamed: 4
0,() () (),NaN,NaN,conditional,NaN
